In [ ]:
import sys
import os
import json

if 'google.colab' in sys.modules:
  print("You are in colab")
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/Otros/arms-robotic-system/jupyter_notebooks')
  !pip install pyngrok drake
  from pyngrok import ngrok

from pydrake.geometry import ( StartMeshcat )
from pydrake.geometry.optimization import (SaveIrisRegionsYamlFile,LoadIrisRegionsYamlFile) # type: ignore

# add parent folders to path
sys.path.append(os.path.abspath('..'))

# Update the import
from src.auxiliar_functions import (VisualizeConnectivity)
from src.gcs_trajectory_optimization import GcsTrajOpt, demo_trajectory_optimization
from src.iris_options import load_config, create_iris_options
from src.joint_teleop import JointTeleop
from src.gcs_helper import VisualizeRegions
from src.end_effector_teleop import EndEffectorTeleop

# Start the visualizer (only run this once).
meshcat = StartMeshcat()
if 'google.colab' in sys.modules:
  # Create a tunnel to the Meshcat server
  ngrok.set_auth_token("2rdws4UBYTqZ1NiiyEbJcWRnByV_7D9Jg8K3YVyZzbFMN1ysg")
  public_url = ngrok.connect(7000, "http")
  print(f"Meshcat is accessible at: {public_url}")

In [ ]:
parent_dir = os.path.abspath('..')
iris_regions = dict()

with open(os.path.join(parent_dir, 'common-files','gcs_seeds.json'), 'r') as f:
    iris_seeds = json.load(f)


if os.path.isfile(os.path.join(parent_dir, 'options.yaml')):
    config = load_config(os.path.join(parent_dir, 'options.yaml'))
    iris_filename = os.path.join(parent_dir,config["iris_regions_path"])
    if os.path.isfile(iris_filename):
        iris_regions.update(LoadIrisRegionsYamlFile(iris_filename))
        print(f"Loaded iris regions from {iris_filename}.")
    else:
        print(f"{iris_filename} not found. No previously computed regions were loaded.")

In [ ]:
with open(os.path.join(parent_dir, 'common-files','gcs_seeds.json'), 'r') as f:
    iris_seeds = json.load(f)
    iris_seeds = {key: iris_seeds[key] for key in iris_regions if key in iris_seeds}

In [ ]:
len(iris_seeds),iris_seeds

In [ ]:
demo_trajectory_optimization(iris_regions, iris_seeds,meshcat)

In [ ]:
VisualizeConnectivity(iris_regions)

In [ ]:
#VisualizeRegions(meshcat, iris_regions)